In [1]:
from luxai_s2.env import LuxAI_S2
from luxai_s2.map.board import Board
from luxai_s2.map_generator.generator import GameMap
import matplotlib.pyplot as plt
import matplotlib
import logging
import numpy as np
from luxai_s2.utils import my_turn_to_place_factory
from agents_experiments import (reset_w_custom_board, IdleAgent, ControlledAgent, interact)
from plots import plot_factory_stats, plot_robot_stats
import uuid
from IPython.display import Video
from pprint import pprint
from robots import CartesianPoint
import pandas as pd
# import seaborn as sns

matplotlib.rc('font', size=14)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# # key-values of dict below should match attributes present in this class
# # https://github.com/aernesto/Lux-Design-S2/blob/adrian/luxai_s2/luxai_s2/config.py#L34
# environment_config = dict(
# #         max_episode_length=350,  # how many total turns in game
# #         CYCLE_LENGTH=50,
# #         DAY_LENGTH=50,  # so no night
#         map_size=48,
#         MAX_RUBBLE=0,  # disable rubble   
#         MIN_FACTORIES=2,  
#         MAX_FACTORIES=5
#     )
# env = LuxAI_S2(**environment_config)
# custom_rubble = np.zeros((env.env_cfg.map_size,) * 2, dtype=np.int64)

# # Let's put a single ice
# custom_ice = np.zeros_like(custom_rubble)
# custom_ice[10, 10] = 1

# # Let's put a single ore at 21, 21
# custom_ore = np.zeros_like(custom_rubble)
# custom_ore[11, 11] = 1

# custom_symmetry = None


# custom_map_ = GameMap(custom_rubble, custom_ice, custom_ore, custom_symmetry)

# simple_board = Board(
#     seed=env.seed_rng.randint(0, 2**32 - 1, dtype=np.int64), 
#     env_cfg=env.env_cfg,
#     existing_map=custom_map_
# )

# obs, env = reset_w_custom_board(env, seed=41, custom_board=simple_board)
    
# # source code of reset() method: 
# # https://github.com/aernesto/Lux-Design-S2/blob/adrian/luxai_s2/luxai_s2/env.py#L175
# # obs = env.reset(seed=41) # resets an environment with a seed

In [3]:
env = LuxAI_S2()
SEED = 1
obs = env.reset(seed=SEED)

In [4]:
logger.setLevel(logging.DEBUG)

filename = str(uuid.uuid4()) + '.mp4'
file_path = '/home/adrian_admin/prog/kaggle/lux-ai-2/Lux-Design-S2/data/experiments/adrian/'
animate_arg = file_path + filename

agents = {}
for player, agent_constructor in zip(env.agents, [IdleAgent, ControlledAgent]):
    logging.debug(player)
    agents[player] = agent_constructor(player, env.state.env_cfg) 
    
last_obs = interact(
    env, 
    agents, 
    10, 
    animate_=animate_arg, 
    break_at_first_action=False, 
    debug=False, 
    custom_board=None,  #simple_board
    seed=SEED
)
# print(last_obs)

DEBUG:root:player_0
DEBUG:root:player_1
DEBUG:root:{x: 31, y: 6, x: 3, y: 35, x: 30, y: 3, x: 46, y: 38, x: 32, y: 6, x: 23, y: 32, x: 28, y: 1, x: 36, y: 3, x: 44, y: 30, x: 30, y: 4, x: 22, y: 33, x: 12, y: 9, x: 34, y: 4, x: 39, y: 8, x: 31, y: 5, x: 10, y: 12, x: 28, y: 2, x: 38, y: 2, x: 33, y: 6, x: 29, y: 3, x: 34, y: 5, x: 44, y: 35, x: 27, y: 1, x: 18, y: 2, x: 29, y: 30, x: 31, y: 4, x: 20, y: 1, x: 33, y: 5, x: 19, y: 2, x: 29, y: 2, x: 31, y: 3, x: 3, y: 36, x: 30, y: 2, x: 29, y: 1, x: 44, y: 36, x: 1, y: 38, x: 25, y: 33, x: 44, y: 29, x: 24, y: 32}
INFO:root:writing /home/adrian_admin/prog/kaggle/lux-ai-2/Lux-Design-S2/data/experiments/adrian/57bf4853-7c08-44f3-af47-ebaa26823c60.mp4


In [5]:
Video(animate_arg, embed=True)

In [6]:
np.array((2, 3, 4)[1:])

array([3, 4])

In [7]:
# #video above is missing bottom pixels for some reason 
# logger.setLevel(logging.INFO)
# img = env.render("rgb_array", width=640, height=640)
# f, ax = plt.subplots(figsize=(7, 7))
# ax.imshow(img);

In [8]:
# logger.setLevel(logging.INFO)
# plot_robot_stats(agents['player_1'].stats);